In [1]:
from pickle import dump, load
from tensorflow.keras.models import Sequential, load_model
import pandas as pd
import numpy as np

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from nltk.stem import PorterStemmer

# Download the stopwords if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from transformers import BertModel, BertTokenizer
import torch

In [4]:
def full_text_cleaning(text):

    pattern = r'[A-Za-z]+'
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()

    text = re.findall(pattern, text)
    text = " ".join(text).lower()
    text = word_tokenize(text)
    text = [i for i in text if i.lower() not in stop_words]

    text = ' '.join([stemmer.stem(i) for i in text])

    return text

In [11]:
tokenizer = load(open('bert_tokenizer.pkl', 'rb'))
bert_model = load(open('bert_model.pkl', 'rb'))

C:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [12]:
def extract_features(texts):

    embeddings = []
    for text in texts:

        inputs = tokenizer.encode(text,
                                  add_special_tokens=True,
                                  return_tensors='pt',
                                  return_truncation=True,
                                  max_length=512)

        with torch.no_grad():
            model_output = bert_model(inputs)
            last_hidden_states = model_output.last_hidden_state

        embeddings.append(last_hidden_states[:, 0 ,:]\
                          .squeeze()\
                          .numpy())

    return np.array(embeddings)

In [31]:
def inference(schema):

    new_data = pd.read_csv('youtube_clean.csv')
    for i in range(10):
        new_data = new_data.sample(frac=1)
    new_data = new_data['text'].head(50)\
                               .apply(lambda x: str(x))
    new_content = new_data.copy()

    new_data = new_data.apply(full_text_cleaning)
    new_data = extract_features(new_data)

    if schema == 'strategy 1':

        model = load_model('model1.h5')
        predictions = model.predict(new_data)
        predicted_classes = np.argmax(predictions, axis=1)

    if schema == 'strategy 2':

        model2 = load(open('model2.pkl', 'rb'))
        predictions = model2.predict(new_data)
        predicted_classes = predictions

    if schema == 'strategy 3':

        model3 = load_model('model1.h5')
        predictions = model3.predict(new_data)
        predicted_classes = np.argmax(predictions, axis=1)

    predicted_df = pd.DataFrame({'text':new_content,
                                 'label':predicted_classes})\
                                 .reset_index(drop=True)\
                                 .replace({0:'Negatif',
                                           1:'Netral',
                                           2:'Positif'})

    return predicted_df

In [34]:
schema = 'strategy 3'
inference(schema)

Keyword arguments {'return_truncation': True} not recognized.
